In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import struct
from array import array
import os

from mnist import load_mnist, export_binary_mnist

# Loading the dataset and pre-processing

In [3]:
X_train, X_test, y_train, y_test = load_mnist(pca=True, n_components=16)

In [4]:
export_binary_mnist(
    X_train,
    y_train,
    "data/train-images-pca16",
    "data/train-labels-binaryMNIST",
)